<a href="https://colab.research.google.com/github/Nwanna-Joseph/QnA/blob/main/question_answering_starter_pack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!rm dev.csv __MACOSX/._dev.csv test.csv __MACOSX/._test.csv train.csv __MACOSX/._train.csv
!wget -O datazip.zip https://github.com/Nwanna-Joseph/QnA/blob/main/qna-data.zip?raw=true
!unzip datazip.zip
!rm datazip.zip

--2022-05-18 23:10:45--  https://github.com/Nwanna-Joseph/QnA/blob/main/qna-data.zip?raw=true
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/Nwanna-Joseph/QnA/raw/main/qna-data.zip [following]
--2022-05-18 23:10:45--  https://github.com/Nwanna-Joseph/QnA/raw/main/qna-data.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Nwanna-Joseph/QnA/main/qna-data.zip [following]
--2022-05-18 23:10:46--  https://raw.githubusercontent.com/Nwanna-Joseph/QnA/main/qna-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1639861

In [27]:
"""
LOAD DATA
"""

import numpy as np 
import pandas as pd 
import json


train_df = pd.read_csv('/content/train.csv', usecols=['item_id','question','review','human_ans_spans']  )
test_df = pd.read_csv('/content/test.csv', usecols=['item_id','question','review','human_ans_spans']  )
# sub_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')

train = np.array(train_df)
test = np.array(test_df)

!mkdir -p data

"""
SETTINGS
"""

use_cuda = True # whether to use GPU or not

In [36]:

train_df.head(10000)

,item_id,question,review,human_ans_spans
0,Yl2TN9c23ZGLUBSD9ks5Uw,How was the service staff?,Came here on a Tuesday evening with a party of...,ANSWERNOTFOUND
1,RtUvSWO_UZ8V3Wpj0n077w,How is staff?,A fun place to go with a few friends. The wai...,ANSWERNOTFOUND
2,N93EYZy9R0sdlEvubu94ig,Is the fill delicious?,I think the best way to describe this place is...,ANSWERNOTFOUND
3,Pthcdg09sHQ9M9Tj5P9gfQ,Is this place big enough?,I've been here a few times and seems to be a h...,ANSWERNOTFOUND
4,nqTvE7ivdU23oUWdI01tOA,What is the price?,This restaurant is truly one of those places w...,$ 11
...,...,...,...,...
2556,a8pmtlVKf7NiSLI-4KejIw,How is music?,We visited this place with visiting family fro...,ANSWERNOTFOUND
2557,tCjlXROlbcaGiBRfU2jaiw,What about place?,Absolutely gorgeous restaurant! This is your s...,Absolutely gorgeous restaurant
2558,rO3lZpVSoRMhhd0AEJBjlg,What is the quality of portion?,After living in Seaton Village/Koreatown South...,ANSWERNOTFOUND
2559,O1TvPrgkK2bUo5O5aSZ7lw,How well served is the dish?,This is the real deal. Amazing variety of dim...,real deal


### Prepare data in QA format

Example-format:

```
train_data = [
    {
        'context': "This tweet sentiment extraction challenge is great",
        'qas': [
            {
                'id': "00001",
                'question': "positive",
                'answers': [
                    {
                        'text': "is great",
                        'answer_start': 43
                    }
                ]
            }
        ]
    }
    ]
```

In [38]:
%%time

"""
Prepare training data in QA-compatible format
"""

# Adpated from https://www.kaggle.com/cheongwoongkang/roberta-baseline-starter-simple-postprocessing

def find_all(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1

def do_qa_train(train):

    output = []
    for line in train:
        context = line[1]

        qas = []
        question = line[-1]
        qid = line[0]
        answers = []
        answer = line[2]
        if type(answer) != str or type(context) != str or type(question) != str:
            print(context, type(context))
            print(answer, type(answer))
            print(question, type(question))
            continue
        answer_starts = find_all(context, answer)
        for answer_start in answer_starts:
            answers.append({'answer_start': answer_start, 'text': answer.lower()})
            break
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})

        output.append({'context': context.lower(), 'qas': qas})
        
    return output

qa_train = do_qa_train(train)

with open('data/train.json', 'w') as outfile:
    json.dump(qa_train, outfile)

CPU times: user 53 ms, sys: 1.16 ms, total: 54.1 ms
Wall time: 54.3 ms


In [37]:
%%time

"""
Prepare testing data in QA-compatible format
"""

def do_qa_test(test):
    output = []
    for line in test:
        context = line[1]
        qas = []
        question = line[-1]
        qid = line[0]
        if type(context) != str or type(question) != str:
            print(context, type(context))
            # print(answer, type(answer))
            print(question, type(question))
            continue
        answers = []
        answers.append({'answer_start': 1000000, 'text': '__None__'})
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})
        output.append({'context': context.lower(), 'qas': qas})
    return output

qa_test = do_qa_test(test)

with open('data/test.json', 'w') as outfile:
    json.dump(qa_test, outfile)

CPU times: user 18 ms, sys: 0 ns, total: 18 ms
Wall time: 19.7 ms


Install [simple-transformers](https://github.com/ThilinaRajapakse/simpletransformers), a tool to train and test transformers model easily.

In [ ]:
!pip install '/kaggle/input/simple-transformers-pypi/seqeval-0.0.12-py3-none-any.whl' -q
!pip install '/kaggle/input/simple-transformers-pypi/simpletransformers-0.22.1-py3-none-any.whl' -q

### Train model

Train the `distilbert-base-uncased-distilled-squad` model

In [ ]:
%%time


from simpletransformers.question_answering import QuestionAnsweringModel

MODEL_PATH = '/kaggle/input/transformers-pretrained-distilbert/distilbert-base-uncased-distilled-squad/'

# Create the QuestionAnsweringModel
model = QuestionAnsweringModel('distilbert', 
                               MODEL_PATH, 
                               args={'reprocess_input_data': True,
                                     'overwrite_output_dir': True,
                                     'learning_rate': 5e-5,
                                     'num_train_epochs': 3,
                                     'max_seq_length': 192,
                                     'doc_stride': 64,
                                     'fp16': False,
                                    },
                              use_cuda=use_cuda)

model.train_model('data/train.json')

### Submission

In [ ]:
%%time

predictions = model.predict(qa_test)
predictions_df = pd.DataFrame.from_dict(predictions)

sub_df['selected_text'] = predictions_df['answer']

sub_df.to_csv('submission.csv', index=False)

print("File submitted successfully.")